In [1]:
# Adjust atomic term energies (SOC matrix diagonals) to fit exptl energy levels
#    to obtain semiempirical term energies
# This version has better assignments, to match theoretical and exptl levels
# ** This program may break if there are multiple terms with the same term symbol **
# ** If so, it can be fixed by adding ordinal prefixes to term symbols            **
# KKI 4/25/2023
import re, sys, glob, subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy, collections
sys.path.insert(0, '../karlib')
import chem_subs as chem
import molpro_subs as mpr

pd.set_option('display.max_rows', None)

In [2]:
# Excel spreadsheet of experimental levels from https://physics.nist.gov/PhysRefData/ASD/levels_form.html
#   Download as CSV; paste into a column in Excel; use Data -> Text to Columns -> Delimited -> Comma
#   Rename that worksheet with a name like "Fe" or "Fe+"
# Note that experimental levels might not be listed by increasing energy
xl_expt = 'exptl_levels.xlsx'
xl = pd.ExcelFile(xl_expt)

### Select atom and parity of interest

In [3]:
atom = 'Ru'  # a name like "Fe" or "Fe+"
parity = 'even'  #  choose 'even' or 'odd' or 'both'

### Select energy maximum for experimental terms

In [4]:
# In case of errors, try making this larger or smaller to match the theoretical calculation

termcut = 12000  # discard terms that lack levels below this energy (cm-1)

In [5]:
Ecol = 'Level (cm-1)'  # the exptl energy column
# display formatting
fmt = {'Eshift': '{:.1f}', Ecol: '{:.3f}', 'Pct': '{:.3f}', 'degen': '{:.0f}'}
for col in ['J', 'Ecalc', 'E_dif', 'Erel', 'Eshift', 'err', 'Eterm', 'cm-1', 'fitted']:
    fmt[col] =  fmt['Eshift']
for col in ['dif', 'Theory', 'ecm', 'SOC', 'RMSE']:
    fmt[col] = '{:.2f}'

In [6]:
def change_prefixes(termlist):
    # Given a list of ASD-type term symbols, like "a 5F",
    #   return a corresonding list of enumerated symbols like "(1)5F"
    # Input list may have repetitions
    # Return the list of new labels
    uniq = [] # maintain ordering (by energy)
    for t in termlist:
        if t not in uniq:
            uniq.append(t)
    nakt = [t.split()[1] for t in uniq]
    olbl = chem.enumerative_prefix(nakt, always=True)
    dmap = dict(zip(uniq, olbl))
    return [dmap[t] for t in termlist]

In [7]:
if atom not in xl.sheet_names:
    print(f'No experimental data sheet for {atom}!')
else:
    dfexpt = pd.read_excel(xl, atom)
    # Delete any ionization limit
    dfexpt = dfexpt[dfexpt.Term != 'Limit']
    print(f'{len(dfexpt)} experimental levels for {atom} read from "{xl_expt}"')
    # Select by parity
    if parity == 'even':
        # discard odd levels ('Term' field ends with '*')
        dfexpt = dfexpt[~dfexpt.Term.str.contains('\*$')]
    elif parity == 'odd':
        dfexpt = dfexpt[dfexpt.Term.str.contains('\*$')]
    print(f'{len(dfexpt)} levels are of parity "{parity}"')
    # Select terms by energy
    lowTerms = []
    for term, grp in dfexpt.groupby('Term'):
        if (grp[Ecol] < termcut).any():
            lowTerms.append(term)
    print(f'There are {len(lowTerms)} assigned terms with levels below {termcut} cm-1:')
    print('   ', lowTerms)
    dfexpt = dfexpt[dfexpt.Term.isin(lowTerms)]
    nlevx = len(dfexpt)
    # parse 'Term' column to get simplified term labels
    def simplify(term):
        # extract the basic LS part of a decorated term label
        regex = re.compile('\d[SPDF-Z]')
        m = regex.search(term)
        if m:
            return m.group(0)
        else:
            # failed
            return '?'
    #dfexpt['Tlbl'] = dfexpt.Term.apply(simplify)  # bare symbols subject to duplication
    dfexpt['Tlbl'] = change_prefixes(dfexpt.Term)  # with numerical enumerative prefixes
    # Convert experimental 'J' and 'Level' to floats
    for col in ['J', Ecol]:
        dfexpt[col] = dfexpt[col].astype(float)
    # add degeneracy = 2J+1
    dfexpt['degen'] = 2 * dfexpt.J + 1
    # sort by energy (just in case)
    dfexpt = dfexpt.sort_values('Level (cm-1)')
    nmagx = int(dfexpt.degen.sum())
    print(f'There are {nlevx} levels of interest ({nmagx} magnetic sublevels)')
    display(dfexpt.style.format(fmt))
    gexpt = list(dfexpt.degen.astype(int))

328 experimental levels for Ru read from "exptl_levels.xlsx"
120 levels are of parity "even"
There are 6 assigned terms with levels below 12000 cm-1:
    ['a 3F', 'a 3P', 'a 5D', 'a 5F', 'a 5P', 'b 3F']
There are 22 levels of interest (126 magnetic sublevels)


,Configuration,Term,J,Prefix,Level (cm-1),Suffix,Uncertainty (cm-1),Lande,Reference,Tlbl,degen
0,4d7.(a 4F).5s,a 5F,5.0,nan,0.000,nan,nan,1.397000,L3466,(1)5F,11
1,4d7.(a 4F).5s,a 5F,4.0,nan,1190.640,nan,nan,1.349000,nan,(1)5F,9
2,4d7.(a 4F).5s,a 5F,3.0,nan,2091.540,nan,nan,1.249000,nan,(1)5F,7
3,4d7.(a 4F).5s,a 5F,2.0,nan,2713.240,nan,nan,1,nan,(1)5F,5
4,4d7.(a 4F).5s,a 5F,1.0,nan,3105.490,nan,nan,0,nan,(1)5F,3
5,4d7.(a 4F).5s,a 3F,4.0,nan,6545.030,nan,nan,1.284000,nan,(1)3F,9
8,4d6.5s2,a 5D,4.0,nan,7483.070,nan,nan,1.447000,nan,(1)5D,9
13,4d7.(a 4P).5s,a 5P,2.0,nan,8043.690,nan,nan,1.536000,nan,(1)5P,5
6,4d7.(a 4F).5s,a 3F,3.0,nan,8084.120,nan,nan,1.196000,nan,(1)3F,7
9,4d6.5s2,a 5D,3.0,nan,8575.420,nan,nan,1.420000,nan,(1)5D,7


### Take assignments at face value, i.e., apply eq. (1)

In [8]:
# No theoretical calculations are needed to use eq. (1)
xterms = []  # list of term labels
eterms = []  # list of term energies
tlbls = []  # simplified term labels
for Term, grp in dfexpt.groupby(['Term']):
    xterms.append(Term)
    emean = np.dot(grp.degen, grp[Ecol]) / grp.degen.sum()
    eterms.append(emean)
    tlbls.append(grp.Tlbl.values[0])
dfeq1 = pd.DataFrame({'Term': xterms, 'Eterm': eterms, 'Tlbl': tlbls}).sort_values('Eterm').reset_index(drop=True)
dfeq1['Erel'] = dfeq1.Eterm - dfeq1.Eterm.min()
print('Simple term energies (cm-1) using eq. (1) and ASD\'s assignments')
display(dfeq1.style.format(fmt))
SOC1 = -1 * np.round(dfeq1.at[0, 'Eterm'], 3)
print(f'The corresponding spin-orbit stabilization energy is SOC1 = {SOC1:.2f} cm-1')

Simple term energies (cm-1) using eq. (1) and ASD's assignments


,Term,Eterm,Tlbl,Erel
0,a 5F,1378.3,(1)5F,0.0
1,a 3F,7686.3,(1)3F,6308.0
2,a 5D,8375.0,(1)5D,6996.7
3,a 5P,8698.4,(1)5P,7320.1
4,b 3F,10185.9,(2)3F,8807.7
5,a 3P,11136.5,(1)3P,9758.2


The corresponding spin-orbit stabilization energy is SOC1 = -1378.26 cm-1


### Specify Molpro SO-CI output file

In [31]:
fsoc = 'fe_15Q21T_ctzdk_x2c.pro'  # use termcut=20000
fsoc = 'fe_ci_15Q7T_c5zdk_x2c.pro'  # use termcut=18000
fsoc = 'fe_15Q7T_ctzdk_x2c.pro'  # use termcut=18000
fsoc = 'Ru_15Q19T_ctz.pro'
fsoc = 'Ru_15Q19T_ctz_valence.pro'
#fsoc = 'Ru_15Q19T_tz_x2c_valence.pro'
fsoc = 'Ru_15Q33T_ctz_valence.pro'

print(f'Reading MOLPRO file "{fsoc}"')
compAtom = mpr.stoichiometry(fsoc)
charge = mpr.total_charge(fsoc)
print(f'The atom is {compAtom} with charge {charge}')
# check for consistency with the experimental data that were read
if charge > 0: 
    compAtom += '+'
elif charge < 0:
    compAtom += '-'
if abs(charge) > 1:
    compAtom += f'{abs(charge)}'
        
if compAtom != atom:
    print(f'*** exptl atom = {atom} is different')
PG = mpr.read_compgroup(fsoc)
print(f'The computational point group is {PG}')

Reading MOLPRO file "Ru_15Q33T_ctz_valence.pro"
The atom is Ru+28 with charge 28
*** exptl atom = Ru is different
The computational point group is Ci


In [32]:
SOCI = mpr.fullmatSOCI(fsoc, hybrid=True, sortval=False)
vals_original = SOCI.vals.copy()

Computational group = Ci
CASSCF states:
    15 Quintet
    33 Triplet


../karlib\molpro_subs.py:499: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[config] = allc


Replacing MRCI+Q energies by HLSDIAG values
*** Warning: "174 SO states but only 132 unique basis states" in parse_SOmatrix()


In [33]:
dfterm = SOCI.average_terms(be_close=['Energy', 'Edav', 'Eref', 'dipZ', 'C0'], always=True)
print('Averaged terms:')
#dfterm['Term'] = chem.enumerative_prefix(dfterm.Term, always=True, style='numeric')
display(dfterm)

Averaged terms:


,Term,dipZ,Edav,idx,ecm
0,(1)5F,0.0,-94.010725,"[0, 1, 2, 3, 4, 5, 6]",0.0
1,(1)5D,0.0,-93.985541,"[7, 8, 9, 10, 11]",5527.1
2,(1)3F,0.0,-93.980966,"[15, 16, 17, 18, 19, 20, 21]",6531.3
3,(1)5P,0.0,-93.971930,"[12, 13, 14]",8514.5
4,(1)3P,0.0,-93.967916,"[22, 23, 24]",9395.4
5,(1)3G,0.0,-93.950070,"[25, 26, 27, 28, 29, 30, 31, 32, 33]",13312.1
6,(2)3P,0.0,-93.947785,"[34, 35, 36]",13813.6
7,(1)3H,0.0,-93.931283,"[37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]",17435.4


In [35]:
def levels_from_term_energies(term_order, term_energies):
    # Install the term energies along the SOCI.matrix diagonal and rediagonalize
    #   'term_order' is array of term symbols
    #   'term_energies' is array of corresponding energies (cm-1)
    # Return nothing
    #global SOCI
    #   'SOCI' is a fullmatSOCI() object and is modified
    term_dict = dict(zip(term_order, term_energies))
    newdiag = SOCI.matrix.diagonal().copy()
    for ibs in range(len(newdiag)):
        j = SOCI.sob_ici[ibs]
        term = SOCI.mrci[j].Term
        # install the new energy for the term
        newdiag[ibs] = term_dict[term]
    # update the matrix
    SOCI.fill_diagonal(newdiag)
    SOCI.diagonalize(store=True, vectors=True, sortval=False)  # update the SOCI() object
    return 

In [36]:
def align_expt_theory(warnThresh=800, showDF=True, silent=False):
    # Compare experimental levels with corresponding theoretical
    #   'warnThresh' will display larger discrepancies in red
    global enforce_term_assignment

    dfdiff = dfexpt.copy()
    dfdiff['Ecalc'] = np.nan
    # match computed levels to exptl
    dflevel = SOCI.dfso
    idx = list(dflevel.index)  # list of computed, assigned levels
    termwt = []  # for arrays of term weights (from theory)
    leading = []  # leading term in the calculation
    failmatch = False
    ifail = []  # list of exptl rows that cannot be matched
    for i, row in dfexpt.iterrows():
        #display(dfexpt.loc[i].to_frame().T)
        while idx:
            # there are theoretical levels that have not been matched to exptl
            for j in idx.copy():
                if float(row.J) != float(dflevel.at[j, 'J']):
                    # values of J don't match
                    continue
                if enforce_term_assignment:
                    # Also require that terms have same label
                    # ** This may be problematic for ambiguous term labeling **
                    if row.Tlbl != dflevel.loc[j, 'Lead']:
                        #print('<<unequal term:', row.Term, dflevel.loc[j, 'Leading'])
                        continue
                # J (and optionally Term) match
                #print('>>>matches:')
                #display(dflevel.loc[j].to_frame().T)
                dfdiff.at[i, 'Ecalc'] = dflevel.at[j, 'Erel']
                termwt.append(dflevel.at[j, 'termwt'])
                leading.append(dflevel.at[j, 'Lead'])
                idx.remove(j)
                break
            else:
                if not silent:
                    print('Failed to assign any theoretical level to this exptl:')
                    display(row.to_frame().T)
                    print('Rejected possibilities:')
                    display(dflevel.loc[idx].style.format(fmt))
                ifail.append(i)
                failmatch = True
            break
    if failmatch:
        if enforce_term_assignment:
            print('*** Try setting enforce_term_assignment = False ***')
        else:
            # just delete the non-matching rows
            if not silent:
                print('Stop tracking these exptl levels:')
                display(dfexpt.loc[ifail])
            dfdiff.drop(ifail, inplace=True)
    dfdiff['err'] = np.round(dfdiff.Ecalc - dfdiff[Ecol], 2)
    try:
        dfdiff['termwt'] = termwt
    except ValueError:
        print('*** Error ***  Try adjusting "termcut" in cell #4 ***')
        1/0
    # keep only some columns
    dfdiff = dfdiff[['Configuration', 'Term', Ecol, 'Tlbl', 'J',
                     'degen', 'Ecalc', 'err', 'termwt']]
    if not enforce_term_assignment:
        # report the theoretical term 
        dfdiff['Leading'] = leading
    if showDF:
        display(dfdiff.drop('termwt', axis=1).style.apply(lambda x: ["background: yellow" if abs(v) > warnThresh else "" for v in x], 
                          subset=pd.IndexSlice[['err']]).format(fmt))
        print(f'Errors > {warnThresh} cm-1 are highlighted')
    return dfdiff

In [37]:
def compute_rmse2(dfexpt, degen=True, DFret=False, silent=False):
    # Given SO-CI level energies return their RMS error
    #   'dfexpt' is the DataFrame of exptl level energies
    #   'degen' is whether to include (2J+1) weights (i.e. sublevels individually)
    #   'DFret' is whether to return the DataFrame
    # Assign values of J using energy and term composition
    #global term_order, SOCI
    SOCI.assign_atomic_J(quiet=True)    
    dfdiff = align_expt_theory(showDF=False, silent=silent)
    dfdiff = dfdiff.sort_values('Level (cm-1)')
    if degen:
        rmse = np.dot(dfdiff.degen, dfdiff.err**2)
        rmse /= dfdiff.degen.sum()
    else:
        rmse = (dfdiff.err**2).mean()
    rmse = np.sqrt(rmse)
    if DFret:
        return rmse, dfdiff
    else:
        return rmse

In [38]:
def rmse_fun(term_energies, silent=False):
    # Uses globals
    # Return RMSE given term energies
    # Do not allow the lowest term energy to change
    #global dfexpt, SOCI, term_order
    levels_from_term_energies(term_order, term_energies)
    rmse = compute_rmse2(dfexpt, silent=silent)
    return rmse
def obj_fun(exc_terme):
    # Given only excited term energies (assuming ground=0)
    #   return the RMSE
    #global dfexpt, SOCI, term_order
    terme = [0] + list(exc_terme)  # the fixed energy of the ground term is 0
    rmse = rmse_fun(terme, silent=True)
    return rmse

In [39]:
# Require that expt and calc have same leading terms? Global variable
enforce_term_assignment = False
if enforce_term_assignment:
    print('--When matching exptl/theor levels, term labels will be required to match--')

In [40]:
# Create global 'term_order' (theoretical terms)
term_order = dfterm.Term.values
print('Term order: ', term_order)
term_energies = dfterm.ecm.values

Term order:  ['(1)5F' '(1)5D' '(1)3F' '(1)5P' '(1)3P' '(1)3G' '(2)3P' '(1)3H']


In [41]:
SOCI.assign_atomic_J(quiet=True)
olddfso = SOCI.dfso.copy()
#olddfso.style.format(fmt)

In [42]:
dferr = align_expt_theory(showDF=True)

,Configuration,Term,Level (cm-1),Tlbl,J,degen,Ecalc,err,Leading
0,4d7.(a 4F).5s,a 5F,0.000,(1)5F,5.0,11,0.0,0.0,(1)5F
1,4d7.(a 4F).5s,a 5F,1190.640,(1)5F,4.0,9,1023.0,-167.6,(1)5F
2,4d7.(a 4F).5s,a 5F,2091.540,(1)5F,3.0,7,1850.6,-240.9,(1)5F
3,4d7.(a 4F).5s,a 5F,2713.240,(1)5F,2.0,5,2468.4,-244.9,(1)5F
4,4d7.(a 4F).5s,a 5F,3105.490,(1)5F,1.0,3,2876.3,-229.2,(1)5F
5,4d7.(a 4F).5s,a 3F,6545.030,(1)3F,4.0,9,5914.3,-630.7,(1)5D
8,4d6.5s2,a 5D,7483.070,(1)5D,4.0,9,6931.1,-552.0,(1)3F
13,4d7.(a 4P).5s,a 5P,8043.690,(1)5P,2.0,5,7348.3,-695.4,(1)5D
6,4d7.(a 4F).5s,a 3F,8084.120,(1)3F,3.0,7,6809.0,-1275.1,(1)5D
9,4d6.5s2,a 5D,8575.420,(1)5D,3.0,7,8152.9,-422.6,(1)3F


Errors > 800 cm-1 are highlighted


In [43]:
rmse_original, dforig = compute_rmse2(dfexpt, DFret=True)  # as received from MOLPRO
oldmat = SOCI.matrix.copy()
oldvals = SOCI.vals.copy()
SOCraw = vals_original[0]
print(f'Original RMSE = {rmse_original:.2f} cm-1 with theoretical SOCraw = {SOCraw:.2f} cm-1')
#display(dforig.style.format(fmt))

Original RMSE = 2056.71 cm-1 with theoretical SOCraw = -1297.50 cm-1


In [44]:
# Summary DF for different data treatments
cols = ['Case', 'RMSE', 'SOC'] + list(dfeq1.Tlbl.values)
xtvals = list(dfeq1.Erel.values)
if not enforce_term_assignment:
    # add any theoretical terms that are missing
    for t in term_order:
        if t not in cols:
            cols.append(t)
            xtvals.append(np.nan)
dfsummary = pd.DataFrame(columns=cols)
for t in (tlbls + list(dfterm.Term)):
    fmt[t] = '{:.1f}'
# situation without theoretical info
row = ['expt only', np.nan, SOC1, *xtvals]
dfsummary.loc[len(dfsummary)] = row

In [45]:
# Use the averaged term energies--expect little change
rmse_avgd = rmse_fun(term_energies)
SOCth = SOCI.dfso.Eshift.min()
print(f'Using averaged input term energies, RMSE = {rmse_avgd:.2f} cm-1 and SOCth = {SOCth:.2f} cm-1')
d = dict(zip(dfterm.Term, dfterm.ecm))
row = ['Before fit', rmse_avgd, SOCth] + [d.get(t, np.nan) for t in dfsummary.columns[3:]]
row.extend([np.nan] * (len(dfsummary.columns) - len(row)))
#display(dfsummary)
#display(row)
dfsummary.loc[len(dfsummary)] = row

Using averaged input term energies, RMSE = 2056.71 cm-1 and SOCth = -1297.50 cm-1


In [46]:
dfsummary.style.format(fmt)

,Case,RMSE,SOC,(1)5F,(1)3F,(1)5D,(1)5P,(2)3F,(1)3P,(1)3G,(2)3P,(1)3H
0,expt only,nan,-1378.26,0.0,6308.0,6996.7,7320.1,8807.7,9758.2,nan,nan,nan
1,Before fit,2056.71,-1297.50,0.0,6531.3,5527.1,8514.5,nan,9395.4,13312.1,13813.6,17435.4


In [47]:
def freport(xvec):
    # callback function to monitor minimization
    freport.counter += 1
    print(f'{freport.counter:5d}', end='')
    return
freport.counter = 0

In [48]:
# Minimize the RMSE
exc_terme = list(term_energies)[1:]  # only excited terms; assume ground term = 0 energy
result = scipy.optimize.minimize(obj_fun, exc_terme, method='Nelder-Mead', callback=freport)

    1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16   17   18   19   20   21   22   23   24   25   26   27   28   29   30   31   32   33   34   35   36   37   38   39   40   41   42   43   44   45   46   47   48   49   50   51   52   53   54   55   56   57   58   59   60   61   62   63   64   65   66   67   68   69   70   71   72   73   74   75   76   77   78   79   80   81   82   83   84   85   86   87   88   89   90   91   92   93   94   95   96   97   98   99  100  101  102  103  104  105  106  107  108  109  110  111  112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127  128  129  130  131  132  133  134  135  136  137  138  139  140  141  142  143  144  145  146  147  148  149  150  151  152  153  154  155  156  157  158  159  160  161  162  163  164  165  166  167  168  169  170  171  172  173  174  175  176  177  178  179  180  181  182  183  184  185  186  187  188  189  190  191  192  193  194  195  196  197  198  199  200

In [55]:
if result.success:
    print(f'Minimization complete in {result.nit} iterations with {result.nfev} evaluations')
    print(f'RMSE = {result.fun:.1f} after Nelder-Mead')

Minimization complete in 350 iterations with 646 evaluations
RMSE = 255.1 after Nelder-Mead


In [86]:
# Compute Hessian 
dE = 20  # step size / cm-1
def grad_fun(xcent):
    # two-sided gradient
    ndim = len(xcent)
    grad = np.zeros(ndim)
    for i in range(ndim):
        x = xcent.copy()
        x[i] += dE
        y1 = obj_fun(x)
        x[i] -= 2 * dE
        y_1 = obj_fun(x)
        grad[i] = (y1 - y_1) / (2 * dE)
    return grad
def hess_fun(xcent):
    # two-sided hessian
    ndim = len(xcent)
    hess = np.zeros((ndim, ndim))
    for i in range(ndim):
        x = xcent.copy()
        x[i] += dE
        y1 = grad_fun(x)
        x[i] -= 2 * dE
        y_1 = grad_fun(x)
        hess[i,:] = (y1 - y_1) / (2 * dE)
    return hess

In [87]:
grad = grad_fun(result.x)
print('Gradient =', np.round(grad, 4))

Gradient = [ 0.0018  0.0012 -0.0005  0.0005  0.0006  0.0006  0.0004]


In [80]:

print('Hessian:')
print(np.round(hess, 6))

Hessian:
[[ 0.0005  0.0001  0.0001  0.     -0.      0.     -0.    ]
 [ 0.0001  0.0005  0.     -0.      0.      0.      0.    ]
 [ 0.0001  0.      0.0003  0.0001 -0.      0.     -0.    ]
 [ 0.     -0.      0.0001  0.0001 -0.      0.      0.    ]
 [-0.      0.     -0.     -0.      0.0004 -0.      0.    ]
 [ 0.      0.      0.      0.     -0.      0.0002 -0.    ]
 [-0.      0.     -0.      0.      0.     -0.      0.    ]]


In [88]:
hess = hess_fun(result.x)
print('Hessian:')
np.set_printoptions(suppress=True)
print(np.round(hess, 5))
print('Diagonal:')
print(np.round(hess.diagonal(), 5))

Hessian:
[[ 0.00054  0.0001   0.00008  0.00002 -0.00001  0.00002 -0.     ]
 [ 0.0001   0.00049  0.00001  0.       0.00003  0.      -0.     ]
 [ 0.00008  0.00001  0.00028  0.00007 -0.00001  0.00002 -0.     ]
 [ 0.00002  0.       0.00007  0.00009 -0.       0.00002  0.     ]
 [-0.00001  0.00003 -0.00001 -0.       0.00042 -0.00001  0.     ]
 [ 0.00002  0.       0.00002  0.00002 -0.00001  0.00019 -0.     ]
 [-0.      -0.      -0.       0.       0.      -0.       0.     ]]
Diagonal:
[0.00054 0.00049 0.00028 0.00009 0.00042 0.00019 0.     ]


In [60]:
# Repeat using BFGS to get an approx Hessian
result2 = scipy.optimize.minimize(obj_fun, result.x, method='BFGS', callback=freport)

In [61]:
if result2.success:
    print(f'Minimization complete in {result2.nit} iterations with {result2.nfev} evaluations')
    print(f'RMSE = {result2.fun:.1f} after BFGS')

Minimization complete in 0 iterations with 8 evaluations
RMSE = 255.1 after BFGS


In [50]:
rmse = result.fun
print(f'After minimization, RMSE = {rmse:.2f} cm-1')
terme = [0] + list(result.x)
Eterm = dict(zip(term_order, terme))
print('\nExptl vs fitted level energies:')
dfdiff = align_expt_theory()
#display(dfcomp.style.format(fmt))
SOCfit = SOCI.vals[0]
print(f'The lowest level energy = SOCfit = {SOCfit:.2f} cm-1')

After minimization, RMSE = 255.14 cm-1

Exptl vs fitted level energies:


,Configuration,Term,Level (cm-1),Tlbl,J,degen,Ecalc,err,Leading
0,4d7.(a 4F).5s,a 5F,0.000,(1)5F,5.0,11,0.0,0.0,(1)5F
1,4d7.(a 4F).5s,a 5F,1190.640,(1)5F,4.0,9,1059.5,-131.1,(1)5F
2,4d7.(a 4F).5s,a 5F,2091.540,(1)5F,3.0,7,1910.1,-181.5,(1)5F
3,4d7.(a 4F).5s,a 5F,2713.240,(1)5F,2.0,5,2536.6,-176.7,(1)5F
4,4d7.(a 4F).5s,a 5F,3105.490,(1)5F,1.0,3,2944.7,-160.8,(1)5F
5,4d7.(a 4F).5s,a 3F,6545.030,(1)3F,4.0,9,6693.4,148.4,(1)3F
8,4d6.5s2,a 5D,7483.070,(1)5D,4.0,9,7657.8,174.8,(1)5D
13,4d7.(a 4P).5s,a 5P,8043.690,(1)5P,2.0,5,8284.3,240.7,(1)5P
6,4d7.(a 4F).5s,a 3F,8084.120,(1)3F,3.0,7,7966.1,-118.1,(1)3F
9,4d6.5s2,a 5D,8575.420,(1)5D,3.0,7,8477.1,-98.3,(1)5D


Errors > 800 cm-1 are highlighted
The lowest level energy = SOCfit = -1365.93 cm-1


In [51]:
d = dict(zip(term_order, terme))
row = ['After fit', rmse, SOCfit] + [d.get(t, np.nan) for t in dfsummary.columns[3:]]
dfsummary.loc[len(dfsummary)] = row

In [52]:
display(dfsummary.style.format(fmt).hide_index())
print(f'Input file: {fsoc}')

Case,RMSE,SOC,(1)5F,(1)3F,(1)5D,(1)5P,(2)3F,(1)3P,(1)3G,(2)3P,(1)3H
expt only,nan,-1378.26,0.0,6308.0,6996.7,7320.1,8807.7,9758.2,nan,nan,nan
Before fit,2056.71,-1297.50,0.0,6531.3,5527.1,8514.5,nan,9395.4,13312.1,13813.6,17435.4
After fit,255.14,-1365.93,0.0,6427.8,7071.4,7728.8,nan,9288.9,7936.7,10000.3,21912.0


Input file: Ru_15Q33T_ctz_valence.pro


In [59]:
result2

      fun: 255.13763075287517
 hess_inv: array([[1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1]])
      jac: array([0., 0., 0., 0., 0., 0., 0.])
  message: 'Optimization terminated successfully.'
     nfev: 8
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([ 7071.4484538 ,  6427.80489763,  7728.79618675,  9288.89299661,
        7936.7176466 , 10000.2921636 , 21911.97197877])